# Tucker-2 decomposition

기존의 Tucker decomposition은 모든 차원에 대해서 decomposition을 수행한다.

하지만 Convolution 커널은 입력 채널, 출력 채널을 제외한 커널사이즈는 일반적으로 크기가 작다는 특징이 있다.

따라서 커널사이즈에 해당하는 dim의 경우 core 크기를 유지하고 나머지에 대해서만 decomposition을 수행한다.

예시로 ($D_H \times D_w \times C_{in} \times C_{out}$) 크기의 텐서에 Tucker-2 decomposition을 수행한다고 가정하자. <br>
이때 $D_H, D_W$는 상대적으로 작은 크기이므로 core tensor에 해당 차원의 크기를 고정한다. <br>
이후 Decomposition은 $C_{in}$과 $C_{out}$을 대상으로 수행되기 때문에 core tensor의 크기는 $D_H \times D_W \times R_3 \times R_4$이 된다.

위의 아이디어를 Conv에 적용하면 다음과 같다.

Conv : $D_H \times D_w \times C_{in} \times C_{out}$ <br>

Decomposition Conv: $1 \times 1 \times C_{int} \times R_3 $ → $D_H \times D_w \times R_3 \times R_4$ → $1 \times 1 \times R_4 \times C_{out}$ 

아래의 코드에서 자세히 알아보겠다. 

[🔗 출처](https://github.com/ruihangdu/Decompose-CNN/blob/master/scripts/torch_tucker.py)

In [44]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import tucker, partial_tucker

def tucker_decomp(layer, rank):
    W = layer.weight.data

    core, [last, first] = partial_tucker(W, modes=[0,1], ranks=rank, init='svd')

    first_layer = nn.Conv2d(in_channels=first.shape[0],
                                       out_channels=first.shape[1],
                                       kernel_size=1,
                                       padding=0,
                                       bias=False)

    core_layer = nn.Conv2d(in_channels=core.shape[1],
                                       out_channels=core.shape[0],
                                       kernel_size=layer.kernel_size,
                                       stride=layer.stride,
                                       padding=layer.padding,
                                       dilation=layer.dilation,
                                       bias=False)

    last_layer = nn.Conv2d(in_channels=last.shape[1],
                                       out_channels=last.shape[0],
                                       kernel_size=1,
                                       padding=0,
                                       bias=True)
    
    if layer.bias is not None:
        last_layer.bias.data = layer.bias.data

    fk = first.t_().unsqueeze_(-1).unsqueeze_(-1)
    lk = last.unsqueeze_(-1).unsqueeze_(-1)

    first_layer.weight.data = fk
    last_layer.weight.data = lk
    core_layer.weight.data = core

    new_layers = [first_layer, core_layer, last_layer]
    return new_layers

먼저 들어오는 Conv layer에 대해 tucker decomposition을 수행한다. <br>
이때 $C_{out}$에 해당하는 컴포넌트는 last에, $C_{in}$에 해당하는 컴포넌트는 first에 저장된다 <br>
이후 채널에 맞게 `nn.Conv2d`를 정의하고 (bias가 있을 경우 bias는 그대로 last_layer에 할당한다.) <br>
tucker를 통해 decompostion을 수행한 텐서들을 각 layer에 할당한다.
